In [5]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [6]:
import sys
sys.path.append("..")
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


First calculate image to text similarities using Clip

In [67]:
import pickle
import torch
!pip install transformers
from transformers import CLIPProcessor, CLIPModel
path = "/content/drive/MyDrive/"
def load_pickle(filename):
    with open(filename, "rb") as input_file:
        object = pickle.load(input_file)
    return object

def create_pickle(filename, object):
    with open(filename, 'wb') as f:
        pickle.dump(object, f)

dict_prompt_captions = load_pickle(path+"dict_question_captions.pickle")
# Now let's 
model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
processor = CLIPProcessor.from_pretrained('openai/clip-vit-large-patch14')
model.cuda()

dict_model_input = load_pickle(path+'dict_model_input_int.pickle')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sim = []
probs = []
with torch.no_grad():
  for i in tqdm(dict_model_input):
      img_ex1, text_ex1, img_ex2, text_ex2, img_q, text_q_prompt = dict_model_input[i]
      
      text_clip = [text_ex1,text_ex2] 
      inputs = processor(text=text_clip, images=img_q, return_tensors="pt", padding=True)
      inputs=inputs.to(device)
      outputs = model(**inputs)

      logits_per_image = outputs.logits_per_image # this is the image-text similarity score
      logits_per_image = logits_per_image.cpu()
      sim.append(logits_per_image.tolist())

      #omit these if you only want cosine similarity
      # if i%100==0:
create_pickle(path+'clip_sim_img2text.pickle',sim)
      

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
100%|██████████| 2500/2500 [02:53<00:00, 14.44it/s]


Now let's calculate img2img similarities using feature extractor

In [57]:
from transformers import ViTImageProcessor, ViTModel

processor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224-in21k') #https://huggingface.co/google/vit-large-patch16-224-in21k
model = ViTModel.from_pretrained('google/vit-large-patch16-224-in21k')
model.cuda()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_sim = []
probs = []
with torch.no_grad():
  for i in tqdm(dict_model_input):
      img_ex1, text_ex1, img_ex2, text_ex2, img_q, text_q_prompt = dict_model_input[i]
      
      images = [img_ex1, img_ex2, img_q]
      inputs = processor(images=images, return_tensors="pt")
      inputs=inputs.to(device)

      outputs = model(**inputs)
      pooler_output = outputs.pooler_output

      scores_1 = torch.nn.functional.cosine_similarity(pooler_output[0], pooler_output[2],dim=0).item()
      scores_2 = torch.nn.functional.cosine_similarity(pooler_output[1], pooler_output[2],dim=0).item()
      img_sim.append((scores_1,scores_2))
      # logits_per_image = outputs.logits_per_image # this is the image-text similarity score
      # logits_per_image = logits_per_image.cpu()
      # img_sim.append(logits_per_image.tolist())

      #omit these if you only want cosine similarity
      # if i%100==0:
      
create_pickle(path+'sim_img2img.pickle',img_sim)

100%|██████████| 2500/2500 [04:42<00:00,  8.86it/s]
